# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Load the csv exported in Part I to a DataFrame
weather_df = pd.read_csv('weather_data.csv')
weather_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,suntar,63,RU,1587799773,66,62.14,117.63,46.60,5.57
1,sibu,75,MY,1587799775,74,2.30,111.82,86.00,6.93
2,laguna,1,US,1587799776,68,38.42,-121.42,69.01,3.38
3,dikson,97,RU,1587799777,97,73.51,80.55,20.28,16.11
4,clyde river,90,CA,1587799778,92,70.47,-68.59,12.20,16.11


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps.
gmaps.configure(api_key=g_key)

# Generate map
fig1 = gmaps.figure(map_type="HYBRID")

# Add locations and weights
locations = weather_df[["Lat", "Lng"]]
weights = weather_df["Humidity"]

# Create heatmap layer
heatmap_layer = gmaps.heatmap_layer(
    locations=locations, weights=weights,
    max_intensity=max(weather_df["Humidity"]), point_radius=5.0
)

# Add locations and weights to map
fig1.add_layer(heatmap_layer)
fig1


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
# Create dataframe to hold only locations with ideal weather conditions

# Filter out temps only between 70-80 degrees F (not inclusive)
best_weather_cities = weather_df.loc[(weather_df["Max Temp"] < 80) & (weather_df["Max Temp"] > 70)]

# Remove cities with wind speed 10 mph or greater
best_weather_cities = best_weather_cities.loc[(best_weather_cities["Wind Speed"] < 10)]

# Remove cities with any cloudiness
best_weather_cities = best_weather_cities.loc[(best_weather_cities["Cloudiness"] == 0)]

best_weather_cities.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
57,arraial do cabo,0,BR,1587799840,75,-22.97,-42.03,70.50,6.82
103,porto murtinho,0,BR,1587799893,75,-21.70,-57.88,73.83,5.19
130,mirante do paranapanema,0,BR,1587799778,60,-22.29,-51.91,71.60,8.05
139,jalu,0,LY,1587799934,30,29.03,21.55,74.91,2.91
219,namibe,0,AO,1587800028,59,-15.20,12.15,77.38,4.70


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
hotel_df = best_weather_cities

hotel_df["Hotel Name"] = ""
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
57,arraial do cabo,0,BR,1587799840,75,-22.97,-42.03,70.50,6.82,
103,porto murtinho,0,BR,1587799893,75,-21.70,-57.88,73.83,5.19,
130,mirante do paranapanema,0,BR,1587799778,60,-22.29,-51.91,71.60,8.05,
139,jalu,0,LY,1587799934,30,29.03,21.55,74.91,2.91,
219,namibe,0,AO,1587800028,59,-15.20,12.15,77.38,4.70,
232,puerto ayacucho,0,VE,1587800043,79,5.66,-67.62,74.68,2.19,
269,victoria point,0,AU,1587800085,64,-27.58,153.30,75.20,6.93,
275,port macquarie,0,AU,1587800092,77,-31.43,152.92,72.00,1.01,
343,sawtell,0,AU,1587800171,69,-30.38,153.10,73.40,9.17,
376,caluquembe,0,AO,1587800209,38,-13.78,14.68,71.74,2.95,


In [6]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
                   
params = {
    "location": "",
    "radius": 5000,
    "type": "hotel",
    "key": g_key
}

#Using Google Places API to find the first hotel for each city located within 5000 meters of coordinates

for index, row in hotel_df.iterrows():
    params["location"] = str(row['Lat']) + "," + str(row['Lng'])
    response = requests.get(base_url, params=params).json()
    results = response['results']
    hotel_df.loc[index,'Hotel Name'] = results[0]['name']
hotel_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
57,arraial do cabo,0,BR,1587799840,75,-22.97,-42.03,70.50,6.82,Cabo Frio
103,porto murtinho,0,BR,1587799893,75,-21.70,-57.88,73.83,5.19,Porto Murtinho
130,mirante do paranapanema,0,BR,1587799778,60,-22.29,-51.91,71.60,8.05,Mirante do Paranapanema
139,jalu,0,LY,1587799934,30,29.03,21.55,74.91,2.91,Jalu
219,namibe,0,AO,1587800028,59,-15.20,12.15,77.38,4.70,Namibe


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [8]:
# Add marker layer ontop of heat map
heatmap_layer = gmaps.heatmap_layer(list(zip(locations['Lat'],locations['Lng'])))
popup_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig = gmaps.figure()

# Adding heatmap layer to the base layer
fig.add_layer(heatmap_layer)

# Add marker layer on top of heat map
fig.add_layer(popup_layer)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))